In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
                database='postgres_db',
                user='postgres_user',
                password='admin',
                host='client.tuple-mia.com',
                port='5432',
                sslmode='require'
            )

In [3]:
temp = conn.cursor()

In [5]:
import yaml
with open('/home/ubuntu/MappingBuffer_event.yml', 'r') as f:
    doc = yaml.load(f)

In [6]:
doc['table_map']

{'CUSTOMER_CONTACT': 'contact',
 'CUSTOMER_MASTER': 'users',
 'CUSTOMER_SECONDARY': 'temp_customer_secondary',
 'EVENT_LOG': 'event',
 'EVENT_MASTER': 'event_master',
 'TRANSACTION_MASTER': 'trans'}

In [7]:
doc['column_map']

{'CUSTOMER_CONTACT': {'cust_id': 'id',
  'email_id': 'email',
  'facebook_id': 'facebook_id',
  'firstname': 'first_name',
  'lastname': 'last_name',
  'phone_number': 'phone'},
 'CUSTOMER_MASTER': {'birthday': 'birthday',
  'create_date': 'create_date',
  'cust_id': 'id'},
 'CUSTOMER_SECONDARY': {'cust_id': 'cust_id'},
 'EVENT_LOG': {'action_type': 'actiontype',
  'cust_id': 'from_id',
  'product_id': 'to_id',
  'timestamp': 'timestamp'},
 'EVENT_MASTER': {'action_category_id': None,
  'action_category_name': None,
  'action_name': 'action_desc',
  'action_type': 'actiontype'},
 'TRANSACTION_MASTER': {'cust_id': 'userid',
  'product_id': 'product_id',
  'quantity': 'quantity',
  'renewal': 'renewal',
  'revenue': 'revenue',
  'timestamp': 'purchase_date'}}

In [8]:
query = "SELECT * FROM %s where %s = ( SELECT %s FROM %s WHERE %s = 'like')"%(
        doc['table_map']['EVENT_LOG'],
        doc['column_map']['EVENT_LOG']['action_type'],
        doc['column_map']['EVENT_MASTER']['action_type'],
        doc['table_map']['EVENT_MASTER'],
        doc['column_map']['EVENT_MASTER']['action_name']        
    )

In [9]:
print query

SELECT * FROM event where actiontype = ( SELECT actiontype FROM event_master WHERE action_desc = 'like')


In [10]:
temp.execute(query)

In [11]:
event = temp.fetchall()

In [12]:
len(event)

2075810

In [13]:
colnames = [desc[0] for desc in temp.description]

In [14]:
colnames

['to_id', 'timestamp', 'actiontype', 'from_id']

In [15]:
import pandas as pd

In [16]:
event_df = pd.DataFrame(event, columns=colnames)

In [17]:
event_df.head()

to_id                 timestamp  actiontype  from_id
0  5891768 2017-02-03 13:41:32+00:00           1  6253829
1  6338583 2017-02-03 13:37:41+00:00           1  6253829
2  6333047 2017-02-03 13:09:40+00:00           1  6257439
3  6273022 2017-02-03 13:32:49+00:00           1  6260574
4   357971 2017-02-03 13:31:10+00:00           1  6270631

In [18]:
event_df.rename(columns={doc['column_map']['EVENT_LOG']['cust_id']:'cust_id'}, inplace = True)
event_df.rename(columns={doc['column_map']['EVENT_LOG']['product_id']:'product_id'}, inplace = True)
event_df.rename(columns={doc['column_map']['EVENT_LOG']['action_type']:'actiontype'}, inplace = True)

In [19]:
event_data = event_df[['cust_id', 'product_id', 'actiontype']]

In [20]:
event_data.head()

cust_id  product_id  actiontype
0  6253829     5891768           1
1  6253829     6338583           1
2  6257439     6333047           1
3  6260574     6273022           1
4  6270631      357971           1

In [21]:
import findspark

In [22]:
findspark.init()

In [23]:
from pyspark import SparkContext
from pyspark.sql import SQLContext,Row
from pyspark.mllib.recommendation import ALS

In [25]:
sc = SparkContext(appName = "BuildProductRecommendations_event")

In [26]:
sqlContext = SQLContext(sc)

In [27]:
n_prod_views_rdd = sqlContext.createDataFrame(event_data).rdd

In [28]:
n_prod_views_rdd.take(5)

[Row(cust_id=6253829, product_id=5891768, actiontype=1),
 Row(cust_id=6253829, product_id=6338583, actiontype=1),
 Row(cust_id=6257439, product_id=6333047, actiontype=1),
 Row(cust_id=6260574, product_id=6273022, actiontype=1),
 Row(cust_id=6270631, product_id=357971, actiontype=1)]

In [29]:
training_rdd, test_rdd = n_prod_views_rdd.randomSplit([8,2], 1345)

In [30]:
training_rdd.distinct().count()

1659644

In [31]:
test_for_predict_rdd = test_rdd.map(lambda x : (x[0], x[1]))

In [32]:
import math

In [33]:
seed = 49247
iterations = 10
lambdas = [0.01, 0.1]
ranks = [16]
alphas = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 40.0, 80.0]
errors = [0 for x in range(len(alphas) * len(ranks) * len(lambdas))]
err_index = 0

In [34]:
for lambda_ in  lambdas:
    for rank in ranks:
        for alpha in alphas:
            model = ALS.trainImplicit(training_rdd, rank, seed=seed, iterations=iterations,
                                           lambda_ = lambda_, alpha = alpha)
            predictions = model.predictAll(test_for_predict_rdd).map(lambda r : ((r[0], r[1]), r[2]))
            views_and_preds = test_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
            error = math.sqrt(views_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
            errors[err_index] = error
            err_index += 1
            print('For rank {0} at alpha: {1} and lambda: {2}, the RMSE is{3}'.format(rank, alpha, lambda_, error))

For rank 16 at alpha: 1.0 and lambda: 0.01, the RMSE is0.757037831509
For rank 16 at alpha: 2.0 and lambda: 0.01, the RMSE is0.708366850511
For rank 16 at alpha: 4.0 and lambda: 0.01, the RMSE is0.644185904052
For rank 16 at alpha: 8.0 and lambda: 0.01, the RMSE is0.568273965949
For rank 16 at alpha: 16.0 and lambda: 0.01, the RMSE is0.487344080658
For rank 16 at alpha: 32.0 and lambda: 0.01, the RMSE is0.410047084848
For rank 16 at alpha: 40.0 and lambda: 0.01, the RMSE is0.387189531648
For rank 16 at alpha: 80.0 and lambda: 0.01, the RMSE is0.324861881164
For rank 16 at alpha: 1.0 and lambda: 0.1, the RMSE is0.769526250568
For rank 16 at alpha: 2.0 and lambda: 0.1, the RMSE is0.714720561119
For rank 16 at alpha: 4.0 and lambda: 0.1, the RMSE is0.646265819466
For rank 16 at alpha: 8.0 and lambda: 0.1, the RMSE is0.569899508045
For rank 16 at alpha: 16.0 and lambda: 0.1, the RMSE is0.490582345093
For rank 16 at alpha: 32.0 and lambda: 0.1, the RMSE is0.412824273893
For rank 16 at alpha

In [35]:
model.recommendProducts(6253829,10)

[Rating(user=6253829, product=6161994, rating=1.1980124516856403),
 Rating(user=6253829, product=6323008, rating=1.1731891727028612),
 Rating(user=6253829, product=504098, rating=1.154091039560083),
 Rating(user=6253829, product=5871783, rating=1.1496445172934815),
 Rating(user=6253829, product=4788525, rating=1.1399174893151376),
 Rating(user=6253829, product=513905, rating=1.1377079663342315),
 Rating(user=6253829, product=5869292, rating=1.1290362942828702),
 Rating(user=6253829, product=3218774, rating=1.1267254489630105),
 Rating(user=6253829, product=5972329, rating=1.1220954283352762),
 Rating(user=6253829, product=5884004, rating=1.1183820321198632)]